### About
This notebook can be used to explore the processing steps in `pudl.analysis.allocate_gen_fuel`

In [ ]:
import pandas as pd

import oge.load_data as load_data

from pudl.analysis import allocate_gen_fuel

year = 2022

In [ ]:
# pick a plant to investigate
plant_to_investigate = 50937

In [ ]:
gf = load_data.load_pudl_table("denorm_generation_fuel_combined_eia923", year)
bf = load_data.load_pudl_table("denorm_boiler_fuel_eia923", year)
gen = load_data.load_pudl_table("denorm_generation_eia923", year)
gens = load_data.load_pudl_table("denorm_generators_eia", year)
bga = load_data.load_pudl_table("boiler_generator_assn_eia860", year)

gf, bf, gen, bga, gens = allocate_gen_fuel.select_input_data(
    gf=gf, bf=bf, gen=gen, bga=bga, gens=gens
)

In [ ]:
bf[bf["plant_id_eia"] == plant_to_investigate]

In [ ]:
gen[gen["plant_id_eia"] == plant_to_investigate]

In [ ]:
gf[gf["plant_id_eia"] == plant_to_investigate]

In [ ]:
freq = "MS"
bf, gens_at_freq, gen = allocate_gen_fuel.standardize_input_frequency(
    bf, gens, gen, freq
)
# Add any startup energy source codes to the list of energy source codes
gens_at_freq = allocate_gen_fuel.adjust_msw_energy_source_codes(gens_at_freq, gf, bf)
gens_at_freq = allocate_gen_fuel.add_missing_energy_source_codes_to_gens(
    gens_at_freq, gf, bf
)

In [ ]:
gens_at_freq

In [ ]:
gens_at_freq[gens_at_freq["plant_id_eia"] == plant_to_investigate]

In [ ]:
gen_assoc = allocate_gen_fuel.associate_generator_tables(
    gens=gens_at_freq, gf=gf, gen=gen, bf=bf, bga=bga
)

In [ ]:
gen_assoc[gen_assoc["plant_id_eia"] == plant_to_investigate]

In [ ]:
gf[gf["plant_id_eia"] == plant_to_investigate]

In [ ]:
### associate generator tables step by step


stack_gens = allocate_gen_fuel.stack_generators(
    gens_at_freq, cat_col="energy_source_code_num", stacked_col="energy_source_code"
).pipe(allocate_gen_fuel.apply_pudl_dtypes, group="eia")
# allocate the boiler fuel data to generators
bf_by_gens = (
    allocate_gen_fuel.allocate_bf_data_to_gens(bf, gens, bga)
    .set_index(allocate_gen_fuel.IDX_GENS_PM_ESC)
    .add_suffix("_bf_tbl")
    .reset_index()
    .pipe(allocate_gen_fuel.apply_pudl_dtypes, group="eia")
)
gf = (
    gf.set_index(allocate_gen_fuel.IDX_PM_ESC)[allocate_gen_fuel.DATA_COLUMNS]
    .add_suffix("_gf_tbl")
    .reset_index()
    .pipe(allocate_gen_fuel.apply_pudl_dtypes, group="eia")
)

In [ ]:
gen_assoc = pd.merge(
    stack_gens,
    gen.rename(columns={"net_generation_mwh": "net_generation_mwh_g_tbl"}),
    on=allocate_gen_fuel.IDX_GENS,
    how="outer",
).merge(
    gf, on=allocate_gen_fuel.IDX_PM_ESC, how="outer", validate="m:1", indicator=True
)

In [ ]:
# identify the complete set of generator ids that match this criteria
retiring_generator_identities = gen_assoc.loc[
    (gen_assoc.operational_status == "retired")
    & (
        (gen_assoc.report_date <= gen_assoc.generator_retirement_date)
        | (gen_assoc.net_generation_mwh_g_tbl.notnull())
    ),
    ["plant_id_eia", "generator_id"],
].drop_duplicates()

In [ ]:
# merge these ids into gen_assoc and keep all months of data for these gens
retiring_generators = gen_assoc.copy().merge(
    retiring_generator_identities,
    how="outer",
    on=["plant_id_eia", "generator_id"],
    indicator="_retiring_gens",
)

retiring_generators = retiring_generators[
    retiring_generators["_retiring_gens"] == "both"
].drop(columns="_retiring_gens")

In [ ]:
retiring_generators[retiring_generators["plant_id_eia"] == plant_to_investigate]

In [ ]:
gen_assoc = gen_assoc.pipe(allocate_gen_fuel.remove_inactive_generators)

In [ ]:
gen_assoc[gen_assoc["plant_id_eia"] == plant_to_investigate]

In [ ]:
unassociated_plants[unassociated_plants["plant_id_eia"] == plant_to_investigate]

In [ ]:
# Generate a fraction to use to allocate net generation and fuel consumption by.
# These two methods create a column called `frac`, which will be a fraction
# to allocate net generation from the gf table for each `IDX_PM_ESC` group
gen_pm_fuel = allocate_gen_fuel.prep_alloction_fraction(gen_assoc)

In [ ]:
gen_pm_fuel[gen_pm_fuel["plant_id_eia"] == plant_to_investigate]

In [ ]:
# Net gen allocation
net_gen_alloc = allocate_gen_fuel.allocate_gen_fuel_by_gen_esc(gen_pm_fuel).pipe(
    allocate_gen_fuel._test_gen_pm_fuel_output, gf=gf, gen=gen
)
allocate_gen_fuel.test_gen_fuel_allocation(gen, net_gen_alloc)

In [ ]:
net_gen_alloc[net_gen_alloc["plant_id_eia"] == plant_to_investigate][
    ["net_generation_mwh"]
].sum()

In [ ]:
gf[gf["plant_id_eia"] == plant_to_investigate][["net_generation_mwh"]].sum()

In [ ]:
gf_923 = load_data.load_pudl_table("generation_fuel_eia923", year)

In [ ]:
gf_923[gf_923["plant_id_eia"] == plant_to_investigate]

In [ ]:
net_gen_alloc[net_gen_alloc["plant_id_eia"] == plant_to_investigate].groupby(
    [
        "report_date",
        "plant_id_eia",
        "prime_mover_code",
        "energy_source_code",
    ]
)["net_generation_mwh"].sum()

In [ ]:
(
    net_gen_alloc[net_gen_alloc["plant_id_eia"] == plant_to_investigate]
    .groupby(
        [
            "report_date",
            "plant_id_eia",
            "prime_mover_code",
            "energy_source_code",
        ]
    )["net_generation_mwh"]
    .sum()
    - gf[gf["plant_id_eia"] == plant_to_investigate]
    .groupby(
        [
            "report_date",
            "plant_id_eia",
            "prime_mover_code",
            "energy_source_code",
        ]
    )["net_generation_mwh"]
    .sum()
).head(50)

In [ ]:
# fuel allocation
fuel_alloc = allocate_gen_fuel.allocate_fuel_by_gen_esc(gen_pm_fuel)

In [ ]:
# check total fuel after allocation
fuel_alloc[fuel_alloc["plant_id_eia"] == plant_to_investigate][
    ["fuel_consumed_mmbtu"]
].sum()

In [ ]:
fuel_alloc[fuel_alloc["plant_id_eia"] == plant_to_investigate]

In [ ]:
# check that allocation fractions sum to 1
fuel_alloc[fuel_alloc["plant_id_eia"] == plant_to_investigate].groupby(
    [
        "report_date",
        "plant_id_eia",
        "prime_mover_code",
        "energy_source_code",
    ]
)["frac"].sum()

In [ ]:
# see where tehre is a difference between inputs and outputs
(
    fuel_alloc[fuel_alloc["plant_id_eia"] == plant_to_investigate]
    .groupby(
        [
            "report_date",
            "plant_id_eia",
            "prime_mover_code",
            "energy_source_code",
        ]
    )["fuel_consumed_mmbtu"]
    .sum()
    - gf[gf["plant_id_eia"] == plant_to_investigate]
    .groupby(
        [
            "report_date",
            "plant_id_eia",
            "prime_mover_code",
            "energy_source_code",
        ]
    )["fuel_consumed_mmbtu"]
    .sum()
)